In [66]:
%pip install azure-ai-ml
%pip install -U 'azureml-rag[faiss]>=0.1.11'
%pip install azureml-core
%pip install azure-identity
%pip install azureml-rag
%pip install azureml.fsspec
%pip install pandas
%pip install openai~=0.27.8 # versioning for to allow dataplane deployment inferring
%pip install python-dotenv
%pip install --upgrade azure-ai-ml
%pip install --upgrade azureml-core

  Obtaining dependency information for azure-storage-blob<13.0.0,>=12.10.0 from https://files.pythonhosted.org/packages/f1/06/68c50a905e1e5481b04a6166b69fecddb87681aae7a556ab727f8e8e6f70/azure_storage_blob-12.17.0-py3-none-any.whl.metadata
  Using cached azure_storage_blob-12.17.0-py3-none-any.whl.metadata (26 kB)
Using cached azure_storage_blob-12.17.0-py3-none-any.whl (388 kB)
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.13.0
    Uninstalling azure-storage-blob-12.13.0:
      Successfully uninstalled azure-storage-blob-12.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.52.0 requires azure-storage-blob<=12.13.0,>=12.5.0, but you have azure-storage-blob 12.17.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached azure_storage_blob-12.13

In [67]:
# If `import win32file` fails with a DLL error then run the following and restart kernel:
# %pip uninstall -y pywin32
# %conda install -y --force-reinstall pywin32

### .env File Setup

Make sure to create a .env file in the same directory as this Jupyter notebook.
The .env file needs to contain the following:

```text
AOAI_API_KEY=<AOAI_API_KEY>
AOAI_ENDPOINT=<AOAI_TARGET_ENDPOINT>
AOAI_API_VERSION=<AOAI_API_VERSION>
```

In [68]:
from os import environ as env
from dotenv import load_dotenv

print("Loading environment variables from .env file")
load_dotenv(".env")

Loading environment variables from .env file


True

### User Input Parameters

Make sure to change the variables in the next section to fit your experiment needs.

In [82]:
# User Input
git_url = 'https://github.com/microsoft/ml-wrappers'
data_source_url = 'https://github.com/microsoft/ml-wrappers'
chunk_size = "1024"
chunk_overlap = "0"
chunk_prepend_summary = False
temperature = "0.5"
max_tokens = "2000"
serverless_instance_count = 1
serverless_instance_type = "Standard_D4s_v3"
embeddings_dataset_name = "ml-wrappers_VectorIndex"

experiment_name = 'qa_faiss_index_generation6'

# RAG Vector Index (and Sample Pipeline) Generation

This notebook shows you how to create a RAG Vector Index from your data (Git repo).

## Get client for AzureML Workspace

The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

In [69]:
# Defaults
registry_name = "azureml"

In [70]:
%%writefile workspace.json
{
    "subscription_id": "ea4faa5b-5e44-4236-91f6-5483d5b17d14",
    "resource_group": "vivianli-rg",
    "workspace_name": "vivianli-ws"
}

Overwriting workspace.json


In [71]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential does not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path='workspace.json')
except Exception as ex:    
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace (associated with the AOAI connection) details."
    ) from ex
ws = Workspace(subscription_id=ml_client.subscription_id, resource_group=ml_client.resource_group_name, workspace_name=ml_client.workspace_name)
print(ml_client)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines 

MLClient(credential=<azure.identity._credentials.browser.InteractiveBrowserCredential object at 0x7fb0a9bf6eb0>,
         subscription_id=ea4faa5b-5e44-4236-91f6-5483d5b17d14,
         resource_group_name=vivianli-rg,
         workspace_name=vivianli-ws)


## Azure OpenAI

We recommend using gpt-35-turbo model or newer to get good quality output. [Follow these instructions](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) to setup an Azure OpenAI Instance and deploy the model. Once you have the model deployed in AOAI you can specify your Model name and Deployment name below.

In [72]:
aoai_completion_model_name = 'gpt-35-turbo'
aoai_completion_deployment_name = 'gpt35turbo'
aoai_embedding_model_name = 'text-embedding-ada-002'
aoai_embedding_deployment_name = 'test'
aoai_connection = "christest123"

In [73]:
from azureml.rag.utils.connections import get_connection_by_name_v2, create_connection_v2

try:
    aoai_connection = get_connection_by_name_v2(ws, aoai_connection)
    aoai_connection_id = aoai_connection['id']
except Exception as ex:
    print(f"Could not get connection '{aoai_connection}', creating a new one")

    target = env['AOAI_ENDPOINT'] # example: 'https://<endpoint>.openai.azure.com/'
    key = env['AOAI_API_KEY']
    apiVersion = env['AOAI_API_VERSION'] # 2023-03-15-preview
    
    if(key is None):
        raise RuntimeError(f"Please provide a valid key for the Azure OpenAI service")
    if(target is None):  
        raise RuntimeError(f"Please provide a valid target for the Azure OpenAI service")
    if(apiVersion is None):
        raise RuntimeError(f"Please provide a valid api-version for the Azure OpenAI service")
    aoai_connection_id = create_connection_v2(
        workspace=ws,
        name=aoai_connection,
        category='AzureOpenAI',
        target=target,
        auth_type='ApiKey',
        credentials={
            'key': key
        },
        metadata={
            'apiType': 'azure',
            'apiVersion': apiVersion
        }
    )['id']

In [74]:
# Uncomment to upgrade azureml-rag if infer_deployment is unrecognized in the package
# %pip install azureml-rag --upgrade

from azureml.rag.utils.deployment import infer_deployment

aoai_completion_deployment_name = infer_deployment(aoai_connection, aoai_completion_model_name)
print(f"Deployment name in AOAI workspace for model '{aoai_completion_model_name}' is '{aoai_completion_deployment_name}'")

Deployment name in AOAI workspace for model 'gpt-35-turbo' is 'gpt35turbo'


### Setup Pipeline

In [91]:
ml_registry = MLClient(credential=credential, registry_name = registry_name)
git_to_faiss_component = ml_registry.components.get('llm_ingest_git_to_faiss_basic', label='latest')

In [84]:
from azure.ai.ml import Output
from azure.ai.ml.dsl import pipeline

# def use_automatic_compute(component, instance_count=1, instance_type='Standard_D4s_v3'):
#     component.set_resources(instance_count=instance_count, instance_type=instance_type, properties={'compute_specification': {'automatic': True}})
#     return component

# def use_aoai_connection(component, aoai_connection_id, custom_env:str=None):
#     if custom_env is not None:
#         component.environment_variables[custom_env] = aoai_connection_id
#     if aoai_connection_id is not None:
#         component.environment_variables['AZUREML_WORKSPACE_CONNECTION_ID_AOAI'] = aoai_connection_id

# @pipeline(compute=dedicated_cpu_compute)
@pipeline(default_compute='serverless')
def qa_faiss_index_generation(
    git_url,
    data_source_url,
    llm_completion_config,
    embeddings_model,
    aoai_connection_id=None,
    chunk_size=1024,
    chunk_overlap=0,
    chunk_prepend_summary=False,
    serverless_instance_count=1,
    serverless_instance_type="Standard_D4s_v3",
    embeddings_dataset_name="git-repository_VectorIndex",
):

    # Ingest Git to Faiss Vector Index
    git_to_faiss = git_to_faiss_component(
        git_repository = git_url,
        data_source_url = data_source_url,
        llm_config = llm_completion_config,
        llm_connection = aoai_connection_id,
        embeddings_model = embeddings_model,
        embedding_connection = aoai_connection_id,
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        chunk_prepend_summary = chunk_prepend_summary,
        serverless_instance_count = serverless_instance_count,
        serverless_instance_type = serverless_instance_type,
        embeddings_dataset_name = embeddings_dataset_name,
    )


    return {
        'qa_faiss_index': git_to_faiss.outputs.faiss_index,
    }

In [85]:
# Defaults
embeddings_model = f'azure_open_ai://deployment/{aoai_embedding_deployment_name}/model/{aoai_embedding_model_name}'
llm_completion_config = f'{{"type":"azure_open_ai","model_name":"{aoai_completion_model_name}","deployment_name":"{aoai_completion_deployment_name}","temperature":"{temperature}","max_tokens":"{max_tokens}"}}'
print(embeddings_model)
print(llm_completion_config)

azure_open_ai://deployment/test/model/text-embedding-ada-002
{"type":"azure_open_ai","model_name":"gpt-35-turbo","deployment_name":"gpt35turbo","temperature":"0.5","max_tokens":"2000"}


In [86]:
from azure.ai.ml import Input
from azure.ai.ml.entities import UserIdentityConfiguration

# data_source_glob=data_source_glob,
# asset_name=asset_name,
# document_path_replacement_regex=document_path_replacement_regex,
pipeline_job = qa_faiss_index_generation(
    git_url = git_url,
    data_source_url = data_source_url,
    llm_completion_config = llm_completion_config,
    embeddings_model = embeddings_model,
    aoai_connection_id=aoai_connection_id,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    chunk_prepend_summary = chunk_prepend_summary,
    serverless_instance_count=serverless_instance_count,
    serverless_instance_type=serverless_instance_type,
    embeddings_dataset_name=embeddings_dataset_name,
)

pipeline_job.identity = UserIdentityConfiguration()
pipeline_job.settings.continue_on_step_failure = False

### Submit Pipeline
Click on the generated link below access the job details on studio. Make sure all necessary flights are added on the URL to access these preview features.

**In case of any errors see [TROUBLESHOOT.md](../../TROUBLESHOOT.md).**

In [87]:
running_pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name=experiment_name
)
running_pipeline_job

Experiment,Name,Type,Status,Details Page
qa_faiss_index_generation6,eager_garden_9lnhhqr2l4,pipeline,Preparing,Link to Azure Machine Learning studio


### Review token usage

In [88]:
# running_pipeline_job = ml_client.jobs.get("<pipeline run id>")
child_runs = ml_client.jobs.list(parent_job_name=running_pipeline_job.name)
child_runs = list(child_runs)
data_generation_run = child_runs[-1]

In [89]:
from azureml.core import Run

run = Run.get(ws, data_generation_run.name)
metrics = run.get_metrics()

In [90]:
print(f"Tokens used: {metrics['total_tokens']}")
print(f"Model used: {metrics['llm_model_name']}")

KeyError: 'total_tokens'

Given the token usage and the model you can compute cost using the pricing here: https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/.